In [1]:
# Importer les librairies nécessaires
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt

## 1. Préparation des données

In [3]:
def load_and_preprocess_data(image_paths, labels):
    images = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (100, 30))  # Redimensionner à 100x30 pixels
        img = img / 255.0  # Normalisation
        images.append(img)
    images = np.array(images).reshape(-1, 30, 100, 1)  # Ajouter la dimension des canaux
    labels = to_categorical(labels, num_classes=36)  # 36 classes (0-9, A-Z)
    return train_test_split(images, labels, test_size=0.2, random_state=42)

# Exemple de données (à remplacer par le dataset ANPR réel)
image_paths = ["ANPR/plate1.jpg", "ANPR/plate2.jpg", "ANPR/plate3.jpg"]  # Remplacez par vos chemins
labels = [123, 456, 789]  # Remplacez par vos étiquettes encodées
X_train, X_test, y_train, y_test = load_and_preprocess_data(image_paths, labels)

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


## 2. Architecture CNN avec 10 couches

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(30, 100, 1)),  # Couche 1
    MaxPooling2D((2, 2)),  # Couche 2
    Conv2D(32, (3, 3), activation='relu'),  # Couche 3
    MaxPooling2D((2, 2)),  # Couche 4
    Conv2D(64, (3, 3), activation='relu'),  # Couche 5
    Conv2D(64, (3, 3), activation='relu'),  # Couche 6
    MaxPooling2D((2, 2)),  # Couche 7
    Flatten(),  # Couche 8
    Dense(128, activation='relu'),  # Couche 9
    Dropout(0.5),  # Couche 10
    Dense(36, activation='softmax')  # Sortie
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 3. Entraînement

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

## 4. Reconnaissance des plaques (sans YOLO)

In [ ]:
def recognize_plate(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (100, 30))
    gray = gray / 255.0
    gray = np.expand_dims(gray, axis=(0, -1))
    prediction = model.predict(gray)
    char_indices = np.argmax(prediction, axis=1)
    chars = [chr(ord('0') + i) if i < 10 else chr(ord('A') + i - 10) for i in char_indices]
    plate_text = ''.join(chars)
    return plate_text

# Exemple avec une vidéo
cap = cv2.VideoCapture("TEST.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_cnn.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

start_time = datetime.now()
license_plates = set()
count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    print(f"Frame Number: {count}")
    plate_text = recognize_plate(frame)
    if plate_text:
        license_plates.add(plate_text)
        cv2.putText(frame, plate_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    out.write(frame)
    cv2.imshow("ANPR with CNN", frame)
    current_time = datetime.now()
    if (current_time - start_time).seconds >= 20:
        save_to_database(license_plates, start_time, current_time)
        start_time = current_time
        license_plates.clear()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

## 5. Sauvegarde dans la base de données

In [ ]:
def save_to_database(license_plates, start_time, end_time):
    conn = sqlite3.connect('licencePlates.db')
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS LicencesPlates (id INTEGER PRIMARY KEY AUTOINCREMENT, start_time TEXT, end_time TEXT, licence_plate TEXT)')
    for plate in license_plates:
        cursor.execute('INSERT INTO LicencesPlates (start_time, end_time, licence_plate) VALUES (?, ?, ?)', (start_time.isoformat(), end_time.isoformat(), plate))
    conn.commit()
    conn.close()

## 6. Évaluation du modèle

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Visualisation
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()